In [7]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

from nltk.tokenize import word_tokenize

In [8]:
dataset=pd.read_csv('spam.csv',encoding = "ISO-8859-1",usecols=[0,1])
x=dataset.iloc[:,1]
y=dataset.iloc[:,0]
y=list(y)


In [9]:
extra_words=list(string.punctuation)+stopwords.words('english')
lemm = WordNetLemmatizer()

In [10]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return 'a'
    elif treebank_tag.startswith('V'):
        return 'v'
    elif treebank_tag.startswith('N'):
        return 'n'
    elif treebank_tag.startswith('R'):
        return 'r'
    else:
        return ''

In [11]:
def clean(data,extra_words):
    data=data.split()
    new_data=[]
   
    for word in data:
        if word  not in extra_words and len(word)>2:
            
            Word=[]
            for w in word:
                if w not in string.punctuation and w.isalpha()==True:
                    Word.append(w)
            word1=''.join(Word)
            word2=word1.lower()  
            if(len(word2))>2 and word.startswith('www')==False:
                try:
                    
                    a=get_wordnet_pos(word2)
                   
                    new_word=lemm.lemmatize(word2,pos=a)
                   
                    new_data.append(new_word)
                
                except:
                    new_data.append(word2)
            else:
                continue
                
            
    return new_data
            
            
            
            
            

In [12]:
def vocab(x):
    document=[]
    for i in x:
        document+=clean(i,extra_words)
    dist = nltk.FreqDist(document)
    vocab_with_frequency = dist.most_common(1500)
    vocab = [i[0] for i in vocab_with_frequency]
    return vocab   

In [13]:
def get_feature(data,vocabulary):
    words=word_tokenize(data)
    #print(type(words))

    feature={}
    for w in vocabulary:
        #print(w)
        feature[w]=words.count(w)
    #print(feature)   
    return feature

In [14]:
vocabulary=list(vocab(x))
new_x=[]
for i in x:
    new_x.append(get_feature(i,vocabulary))

In [15]:
data=[]
for i in range(len(new_x)):
    data.append((new_x[i],y[i]))


In [16]:
import random
random.shuffle(data)

In [17]:
training_data=data[:3801]
testing_data=data[3801:]

In [18]:
clf = nltk.NaiveBayesClassifier.train(training_data)
clf.show_most_informative_features(10)

Most Informative Features
                     txt = 1                spam : ham    =     91.0 : 1.0
                   apply = 1                spam : ham    =     88.0 : 1.0
                landline = 1                spam : ham    =     79.5 : 1.0
                 network = 1                spam : ham    =     75.2 : 1.0
                    draw = 1                spam : ham    =     70.9 : 1.0
                customer = 1                spam : ham    =     65.7 : 1.0
                    rate = 1                spam : ham    =     62.3 : 1.0
                 anytime = 1                spam : ham    =     53.7 : 1.0
                   video = 1                spam : ham    =     53.6 : 1.0
                     win = 1                spam : ham    =     52.8 : 1.0


In [19]:
nltk.classify.accuracy(clf, testing_data)

0.9723320158102767

In [20]:
new_x=pd.DataFrame(new_x)
new_x.shape

(5572, 1500)

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
train_x,test_x,train_y,test_y=train_test_split(new_x,y,test_size=0.2,random_state=1)

In [23]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB() #call the method
naive_bayes.fit(train_x, train_y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [24]:
predictions = naive_bayes.predict(test_x)

In [25]:
naive_bayes.score(test_x,test_y)

0.9650224215246637